In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('Spark Prac').setMaster('local')
sc = SparkContext(conf = conf)

In [52]:
data = sc.textFile('Admission_Prediction.csv')

#### Import PySpark ML Libraries

In [53]:
from __future__ import print_function
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [54]:
spark = SparkSession.builder.appName("Spark ML Algo").getOrCreate()

In [55]:
dataframe = spark.read.csv("Admission_Prediction.csv", header=True)

In [56]:
type(dataframe)

pyspark.sql.dataframe.DataFrame

In [57]:
dataframe.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     NULL|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             NULL|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [13]:
dataframe.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [14]:
from pyspark.sql.functions import col
new_dataframe = dataframe.select(*(col(c).cast("float").alias(c) for c in dataframe.columns))

In [15]:
new_dataframe.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [16]:
from pyspark.sql.functions import col, count, isnan, when

In [17]:
for c in new_dataframe.columns:
    print(c)

GRE Score
TOEFL Score
University Rating
SOP
LOR
CGPA
Research
Chance of Admit


In [18]:
### Checking for null ir nan type values in out columns
new_dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataframe.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [19]:
from pyspark.ml.feature import Imputer

In [20]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_dataframe)

imputed_data = model.transform(new_dataframe)

In [21]:
imputed_data

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float, Chance of Admit: float]

In [22]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [23]:
features = imputed_data.drop('Chance of Admit')

In [24]:
features

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float]

In [25]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler( inputCols=features.columns,outputCol="features")

In [26]:
output = assembler.transform(imputed_data)

#### Linear Regressor

In [27]:
output= output.select("features", "Chance of Admit")

In [28]:
output = assembler.transform(imputed_data)

In [29]:
output= output.select("features", "Chance of Admit")

In [30]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [31]:
train_df.show()
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,98.0,2.0,4...|           0.34|
|[298.0,99.0,1.0,1...|           0.53|
+--------------------+---------------+
only showing top 20 rows

+--------------------+---------------+

In [32]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit')
linear_model = lin_reg.fit(train_df)

In [33]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [0.0021583914573786995,0.002040306041177352,0.009739643411786267,0.0018670741847619792,0.015687437262470957,0.1173172682669684,0.023649461519646545]
Intercept: -1.2921313963633154


In [34]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.060481
r2: 0.813760


In [35]:
# prediction

predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit","features").show()

+------------------+---------------+--------------------+
|        prediction|Chance of Admit|            features|
+------------------+---------------+--------------------+
|0.4686400904965997|           0.49|[294.0,95.0,1.0,1...|
|0.4552270031228267|           0.47|[295.0,96.0,2.0,1...|
|0.5152707791434039|           0.57|[295.0,99.0,2.0,2...|
|0.5541659713956584|           0.61|[296.0,99.0,2.0,2...|
|0.5180916055732037|           0.43|[297.0,96.0,2.0,2...|
|0.5741575223697288|           0.57|[297.0,101.0,3.0,...|
|0.5432479239887194|           0.38|[299.0,97.0,3.0,5...|
|0.5268066136873575|           0.59|[299.0,100.0,1.0,...|
|0.6642069620214324|           0.56|[299.0,102.0,3.0,...|
|0.5479129953497863|           0.58|[300.0,99.0,1.0,1...|
|0.4254989875293911|           0.36|[300.0,99.0,1.0,3...|
|0.5404389175111872|           0.56|[300.0,102.0,2.0,...|
|0.5276744821542532|           0.58|[300.0,105.0,1.0,...|
|0.5757064883564009|           0.44|[301.0,97.0,2.0,3...|
|0.58033809076

In [36]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit",metricName="r2")
print("R Squared (R2) on test data =", pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.8352227680525521


#### Random Forest Regressor

In [37]:
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

In [38]:
featureIndexer = featureIndexer.transform(output)

In [39]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit")

In [40]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

In [41]:
training.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,98.0,2.0,4...|           0.34|
+--------------------+---------------+
only showing top 20 rows



In [42]:
test.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[295.0,101.0,2.0,...|           0.69|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,2...|           0.61|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,100.0,1.0,...|           0.52|
|[298.0,99.0,1.0,1...|           0.53|
|[299.0,97.0,3.0,5...|           0.38|
|[299.0,100.0,1.0,...|           0.59|
|[299.0,100.0,3.0,...|           0.63|
|[300.0,95.0,2.0,3...|           0.62|
|[300.0,97.0,2.0,3...|           0.65|
|[300.0,98.0,1.0,2...|           0.61|
|[300.0,99.0,1.0,1...|           0.58|
|[300.0,101.0,3.0,...|           0.59|
|[300.0,102.0,3.0,...|           0.63|
|[301.0,96.0,1.0,3...|           0.54|
|[301.0,98.0,1.0,2...|           0.67|
|[301.0,99.0,2.0,3...|           0.64|
|[301.0,100.0,3.0,...|           0.67|
|[302.0,99.0,2.0,1...|           0.56|
+--------------------+---------------+
only showing top 20 rows



In [43]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit" )

In [44]:
# Train model.  This also runs the indexer.
model = random_forest_reg.fit(test)

In [45]:
# Make predictions.
predictions = model.transform(test)

In [46]:
predictions.show()

+--------------------+---------------+------------------+
|     indexedFeatures|Chance of Admit|        prediction|
+--------------------+---------------+------------------+
|[295.0,101.0,2.0,...|           0.69| 0.583469777743509|
|[296.0,97.0,2.0,1...|           0.49|0.5425059525084875|
|[296.0,99.0,2.0,2...|           0.61|0.6239952150053376|
|[297.0,98.0,2.0,2...|           0.59|0.5427751646312526|
|[297.0,100.0,1.0,...|           0.52|0.5793593166838861|
|[298.0,99.0,1.0,1...|           0.53|0.5208974595634693|
|[299.0,97.0,3.0,5...|           0.38|0.4417110356933627|
|[299.0,100.0,1.0,...|           0.59|0.5793593166838861|
|[299.0,100.0,3.0,...|           0.63|0.6165215220904843|
|[300.0,95.0,2.0,3...|           0.62|0.6267901502373723|
|[300.0,97.0,2.0,3...|           0.65|0.6346901490469825|
|[300.0,98.0,1.0,2...|           0.61|0.6248327365868962|
|[300.0,99.0,1.0,1...|           0.58|0.5819419980631088|
|[300.0,101.0,3.0,...|           0.59|0.5750301354573739|
|[300.0,102.0,

In [47]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit", predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data =  0.043059444344646834


In [48]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit", predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.900575696554627
